# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.conda_dependencies import CondaDependencies

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'student-math-experiment-automl'

experiment=Experiment(ws, experiment_name)

# Setting up the enviornment
env = Environment.get(workspace=ws, name="my-captured-notebook-env")

deps = CondaDependencies()
deps.add_pip_package("pandas")
deps.add_pip_package("numpy")
deps.add_pip_package("joblib")
deps.add_pip_package("azureml-defaults")
deps.add_pip_package("azureml-train-automl-runtime")

env.python.conda_dependencies = deps

In [4]:
from azureml.core import Dataset

# Load dataset from Azure ML workspace
df = Dataset.get_by_name(workspace=ws, name="student-math")



## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,  
    "primary_metric": "normalized_mean_absolute_error",  
    "n_cross_validations": 5, 
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="regression",  
    training_data=df,  
    label_column_name="G3", 
    compute_target="notebook-compute", 
    enable_early_stopping=True,
    featurization="auto",  
    **automl_settings
) 

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
student-math-experiment-automl,AutoML_c47569dc-eceb-4e2b-a792-bdc1acfa69ee,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
student-math-experiment-automl,AutoML_c47569dc-eceb-4e2b-a792-bdc1acfa69ee,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

************************************************************

{'runId': 'AutoML_c47569dc-eceb-4e2b-a792-bdc1acfa69ee',
 'target': 'notebook-compute',
 'status': 'Completed',
 'startTimeUtc': '2025-03-24T16:41:19.944449Z',
 'endTimeUtc': '2025-03-24T16:58:56.503407Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_mean_absolute_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'notebook-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"student-math-experiment-automl","subscription_id":"48778e11-0fc7-4fc8-a16c-304a430e61a4","resource_group":"student-performance-rg","workspace_name":"student-performance-ws","region":"n

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails

# Show run details in the notebook
RunDetails(remote_run).show()

2025-03-24 16:59:38.026967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-24 16:59:38.985511: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-24 16:59:39.263755: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 16:59:41.448952: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-24 16:59:46.729575: W tensorflow/compiler/tf2

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
best_run, best_model = remote_run.get_output()

In [14]:
import joblib

#TODO: Save the best model
print("Best Run ID:", best_run.id)
print("Best Model Properties:")
for key, value in best_run.get_metrics().items():
    print(f"{key}: {value}")


model_filename = "best_automl_model.pkl"
joblib.dump(best_model, model_filename)

Best Run ID: AutoML_c47569dc-eceb-4e2b-a792-bdc1acfa69ee_30
Best Model Properties:
r2_score: 0.8188106270801259
normalized_root_mean_squared_log_error: 0.15381860999667846
normalized_median_absolute_error: 0.03798832359611608
explained_variance: 0.8196532389297546
root_mean_squared_error: 1.8975014257841507
mean_absolute_error: 1.2243031615041788
normalized_mean_absolute_error: 0.06121515807520894
mean_absolute_percentage_error: 9.72984100358316
root_mean_squared_log_error: 0.46830420947431595
median_absolute_error: 0.7597664719223214
normalized_root_mean_squared_error: 0.09487507128920752
spearman_correlation: 0.9402797027102686
residuals: aml://artifactId/ExperimentRun/dcid.AutoML_c47569dc-eceb-4e2b-a792-bdc1acfa69ee_30/residuals
predicted_true: aml://artifactId/ExperimentRun/dcid.AutoML_c47569dc-eceb-4e2b-a792-bdc1acfa69ee_30/predicted_true


['best_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [10]:
from azureml.core.model import Model
import os

# Ensure the model file exists
model_file = "best_automl_model.pkl"
if not os.path.isfile(model_file):
    raise FileNotFoundError(f"The model file {model_file} does not exist.")

# Register the model
model = Model.register(
    workspace=ws,
    model_name="student-performance-model",
    model_path=model_file,
    description="AutoML trained model for predicting student performance",
    tags={
        "type": "AutoML",
        "task": "regression", 
        "framework": "scikit-learn"
    }
)

print(f"Model registered: Name={model.name}, ID={model.id}, Version={model.version}")


Registering model student-performance-model
Model registered: Name=student-performance-model, ID=student-performance-model:4, Version=4


In [11]:
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import LocalWebservice

service_name = 'student-performance-predictor'

inference_config = InferenceConfig(entry_script='score.py', environment=env)
aci_config =  LocalWebservice.deploy_configuration(port=6789)

# Deploying Serice
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],  
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)

service.wait_for_deployment(show_output=True)

print(f"Scoring URI: {service.scoring_uri}")



Generating Docker build context.
======Starting Image Build on Serverless Compute======
The run ID for the image build on serverless compute is imgbldrun_aaace0d
Additional logs for the run: https://ml.azure.com/experiments/id/prepare_image/runs/imgbldrun_aaace0d?wsid=/subscriptions/48778e11-0fc7-4fc8-a16c-304a430e61a4/resourcegroups/student-performance-rg/workspaces/student-performance-ws&tid=f804f881-90d8-4e7d-8309-7fb565eaaf2c
2025-03-24T17:03:31: Logging into Docker registry: f72329ad78ea468eb9cb3c2ed2292a9d.azurecr.io
2025-03-24T17:03:31: WARNING! Using --password via the CLI is insecure. Use --password-stdin.
2025-03-24T17:03:31: WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
2025-03-24T17:03:31: Configure a credential helper to remove this warning. See
2025-03-24T17:03:31: https://docs.docker.com/engine/reference/commandline/login/#credentials-store

2025-03-24T17:03:31: Login Succeeded


2025-03-24T17:03:31: Running: ['docker', 'build', '-f', 'a

ERROR:azureml._model_management._util:Error: Container has crashed. Did your init method fail?



WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
input_data = {
    "data": [
        {
            "school": "GP", "sex": "F", "age": 16, "address": "R",
            "famsize": "LE3", "Pstatus": "T", "Medu": 2, "Fedu": 3,
            "Mjob": "other", "Fjob": "teacher", "reason": "home",
            "guardian": "father", "traveltime": 3, "studytime": 3,
            "failures": 2, "schoolsup": "yes", "famsup": "yes",
            "paid": "yes", "activities": "no", "nursery": "yes",
            "higher": "no", "internet": "yes", "romantic": "no",
            "famrel": 1, "freetime": 2, "goout": 4, "Dalc": 5,
            "Walc": 5, "health": 1, "absences": 8, "G1": 15, "G2": 20
        },
        {
            "school": "GP", "sex": "M", "age": 16, "address": "U",
            "famsize": "GT3", "Pstatus": "A", "Medu": 2, "Fedu": 0,
            "Mjob": "health", "Fjob": "health", "reason": "home",
            "guardian": "other", "traveltime": 3, "studytime": 2,
            "failures": 1, "schoolsup": "no", "famsup": "yes",
            "paid": "yes", "activities": "yes", "nursery": "no",
            "higher": "no", "internet": "yes", "romantic": "no",
            "famrel": 1, "freetime": 4, "goout": 4, "Dalc": 1,
            "Walc": 2, "health": 1, "absences": 18, "G1": 8, "G2": 10
        },
        {
            "school": "GP", "sex": "M", "age": 20, "address": "U",
            "famsize": "LE3", "Pstatus": "T", "Medu": 2, "Fedu": 0,
            "Mjob": "teacher", "Fjob": "other", "reason": "reputation",
            "guardian": "other", "traveltime": 3, "studytime": 1,
            "failures": 0, "schoolsup": "no", "famsup": "yes",
            "paid": "yes", "activities": "no", "nursery": "yes",
            "higher": "yes", "internet": "no", "romantic": "yes",
            "famrel": 1, "freetime": 2, "goout": 2, "Dalc": 3,
            "Walc": 2, "health": 1, "absences": 11, "G1": 17, "G2": 13
        }
    ]
}

In [ ]:
import requests
import json

# Replace with your actual endpoint URL
endpoint_url = "https://your-endpoint-url.azurewebsites.net/score"

# If your service requires authentication, include the key
api_key = "your-api-key-if-required"  # Replace with actual key if needed
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}" if api_key else None
}

# Send request to the web service
response = requests.post(endpoint_url, data=json.dumps(input_data), headers=headers)

# Print response
print("Prediction Response:", response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
from azureml.core.webservice import Webservice

# Print logs
print("Service logs:")
print(service.get_logs())

# Delete the service
service.delete()
print(f"Service '{service_name}' has been deleted.")

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
